# 1 - Présentation
## 1.1 - Introduction
Les réseaux sociaux sont une source de données presques infinis. Avec les Tweets sur Twitter ou les murs sur Facebook, il y a une multitude d'informations en attente d'être étudiées.

L'objectif de ce jeu de données est de prédire si un tweet est positif ou négatif. L'exercice nécessite de réaliser une analyse de sentiment en réalisant une classification à 2 sorties.

## 1.2 - Source de Données
Les données sont représentés par 2 dimensions :
* un chiffre de 0 ou 4 qui représente si un tweet est positif(4) ou négatif(0) ;
* une chaine de caractères représentant le tweet.

# 2 - Préparation des données
## 2.1 - Charger les librairies
Ci-dessous la liste des librairies nécessaires durant l'analyse des données :

In [ ]:
import pandas as pd
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
dataSource = pd.read_csv('./data/training.csv', 
                 header=0,
                 sep=',',
                 names=['Sentimental','Id','Date','Query', 'Owner','Tweet'],
#                 nrows = 5000000,
                 encoding = 'latin-1'
)

Maintenant nous allons afficher les 20 premières lignes de ce dataset pour mieux appréhender l'information :

In [ ]:
dataNegative = dataSource[-10000:]
dataPositive = dataSource[:10000]
 
df = pd.concat([dataPositive, dataNegative])

df.head(20)

Il  y a 6 dimensions dans ce dataset :
* Le sentiment du tweet (0 = negatif, 2 = neutre et 4 = positif) ;
* L'id du tweet ;
* La date de publication ;
* La requête. (S'il n'y en a aucune alors NO_QUERY) ;
* L'utilisateur qui a tweeté ;
* Le texte du tweet.

Nous allons afficher les types des données chargées :

In [ ]:
df.dtypes

Les types des données sont corrects pour l'analyse. Il n'est pas nécessaire de les casts.

## 2.3 - Nettoyer les données
Dans l'objectif d'analyser les données, il faut tout d'abord les nettoyer afin qu'elles soient plus facile à lire pour la machine.

Durant cette partie nous allons définir des règles pour le nettoyage de données non structurée au format texte.

In [ ]:
def dataClean(column):
    signs = ['.', ',' ,'!', '?', ';', "'", '-', '_', '&', '(', ')', '*']
    
    column = column.str.lower()
    column = column.str.replace('@[\w]*', '')
    column = column.str.replace('#[\w]*', '')
    
    for sign in signs:
        column = column.str.replace(sign, '')
    
    column = column.str.replace('/^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$/', '')
    column = column.str.strip()
    
    return column
    
df['Tweet'] = dataClean(df['Tweet'])

Cette fonction va permettre de préparer les données pour la suite de l'analyse.

Les actions réalisées :
* Mettre le texte en minuscule afin d'éviter de compter certains mots en double comme par exemple : Hasard et hasard. C'est le même mot pour nous en le lisant mais pour l'algorithme cela est différent ;
* Supprimer les pseudos Twitter ;
* Supprimer les caractères comme le "." ou le "!" ;
* Supprimer les URL ;
* Supprimer les espaces avant et après le paragraphe.

## 2.4 - Preprocessing
Maintenant que nous avons nettoyer les données, il faut adapter l'information afin de la rendre exploitable par les algorithmes de Machine Learning.

In [ ]:
def preprocessing(column):
    wordnet_lemmatizer = WordNetLemmatizer()
    column = column.apply(lambda row: nltk.word_tokenize(row))
    column = column.apply(lambda x: [item for item in x if item not in stopwords.words('english')])
    column = column.apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x])

    return column
df['TweetTK'] = preprocessing(df['Tweet'])    

Cette fonction va permettre de **Tokenize** les tweets, faire disparaitre les **Stop Words** puis d'appliquer un algorithme **Lemmatisation**.

## 2.5 - Bag of Words

In [ ]:
def getListofWords(tweets):
    allWords = []
    for words in tweets:
        allWords.extend(words)
    return allWords

def getWordFeatures(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def findFeatures(document, wordFeatures):
    features = {}
    for w in wordFeatures:
        features[w] = (w in document)
    return features

listWords = getListofWords(df['TweetTK'])
wordFeatures = getWordFeatures(listWords)
df['TweetFeatures'] = df['TweetTK'].apply(lambda x: findFeatures(x, wordFeatures))

## 2.6 - Conclusion
Les données sont maintenant exploitables pour commencer l'étape d'apprentissage en Machine Learning.

# 3. Apprentissage
Dans cette partie nous allons apprendre à l'algorithme NaiveBayes à reconnaitre si un tweet est positif ou négatif.

In [ ]:
trainingSet = []

for words, sentimental in zip(df['TweetTK'], df['Sentimental']):
    for word in words:
        trainingSet.append(({'word': word}, sentimental))

classifier = nltk.NaiveBayesClassifier.train(trainingSet)

# 4. Vérification
Dans cette partie, nous allons vérifier la qualité de l'apprentissage sur un jeu de données de test.

In [ ]:
datasetTest = pd.read_csv('./data/test.csv', 
                 header=0,
                 sep=',',
                 names=['Sentimental','Id','Date','Query', 'Owner','Tweet'],
                 encoding = 'latin-1'
)

datasetTest['Tweet'] = dataClean(datasetTest['Tweet'])
datasetTest['TweetTK'] = preprocessing(datasetTest['Tweet'])

datasetTest.head()

listWords = getListofWords(datasetTest['TweetTK'])
wordFeatures = getWordFeatures(listWords)
datasetTest['TweetFeatures'] = datasetTest['TweetTK'].apply(lambda x: findFeatures(x, wordFeatures))

testSet = []

for words, sentimental in zip(datasetTest['TweetTK'], datasetTest['Sentimental']):
    for word in words:
        testSet.append(({'word': word}, sentimental))



Nous allons maintenant vérifier le résultat sur un dataset de test pour connaitre le % de bonne réponses.

Le taux de bonnes réponses est d'environ 42% avec 20000 tweets. Malheureusement l'ordinateur qui fait tourner se script ne peut pas prendre en charge les 1 600 000 tweets de la base de tests avec cet algorithme.

In [ ]:
classifier.show_most_informative_features(15)

In [ ]:
testSet

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testSet))*100)

# 5. Conclusion

L'algorithme de ML Naive Bayes à l'air de fonctionner malheureusement, il est possible d'utiliser le dataset complet pour l'apprentissage et voir le résultat.